In [3]:
import pyodbc
import pandas as pd
import warnings

from sqlalchemy import create_engine
import urllib

warnings.filterwarnings("ignore", category=UserWarning)


In [4]:
import os
senha = os.getenv('senha_bd_simples')


conn = pyodbc.connect(
    "DRIVER={SQL Server};"\
    "SERVER=den1.mssql7.gear.host;"\
    "DATABASE=vendasbicyclon;"\
    "UID=vendasbicyclon;"\
    "PWD=senha@clarify;" \
    'MARS_Connnection=yes;'
)

In [38]:
def exe_query(query):
    return pd.read_sql(query, conn)

lista_Resposta = []
def exe_varias_querys(*args):
    for consulta in args:
        lista_Resposta.append(pd.read_sql(consulta, conn))
    return lista_Resposta


In [31]:
q = """
    select * from clientes
"""
 
exe_query(q)

,idCliente,nomeCliente,cidadeCliente
0,1,Leonardo Alves,Rio de Janeiro
1,2,Ana Clara Rodrigues,Rio de Janeiro
2,3,Felipe Gomes Santos,São Paulo
3,4,Camila Oliveira Lima,São Paulo
4,5,Thiago Costa Ferreira,Belo Horizonte
5,6,Beatriz Martins Rocha,Belo Horizonte
6,7,Gabriel Souza Lima,Curitiba
7,8,Larissa Andrade Melo,Curitiba
8,9,Ricardo Henrique Silva,Salvador
9,10,Juliana Pereira Costa,Salvador


In [32]:
q1 ="select * from clientes"
q2 = "select * from produtos"
 
exe_varias_querys(q1,q2)
 

[    idCliente              nomeCliente   cidadeCliente
 0           1           Leonardo Alves  Rio de Janeiro
 1           2      Ana Clara Rodrigues  Rio de Janeiro
 2           3      Felipe Gomes Santos       São Paulo
 3           4     Camila Oliveira Lima       São Paulo
 4           5    Thiago Costa Ferreira  Belo Horizonte
 5           6    Beatriz Martins Rocha  Belo Horizonte
 6           7       Gabriel Souza Lima        Curitiba
 7           8     Larissa Andrade Melo        Curitiba
 8           9   Ricardo Henrique Silva        Salvador
 9          10    Juliana Pereira Costa        Salvador
 10         11     Eduardo Alves Mendes        Brasília
 11         12   Carolina Nunes Ribeiro        Brasília
 12         13   Matheus Gonçalves Lima          Recife
 13         14     Amanda Barbosa Souza          Recife
 14         15      Bruno Dias Carvalho       Fortaleza
 15         16    Isabela Freitas Rocha       Fortaleza
 16         17  Leonardo Castro Almeida    Porto

In [33]:
params = urllib.parse.quote_plus(
     "DRIVER={SQL Server};"\
    "SERVER=den1.mssql7.gear.host;"\
    "DATABASE=vendasbicyclon;"\
    "UID=vendasbicyclon;"\
    "PWD=senha@clarify;" \
    'MARS_Connnection=yes;'
)
 
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={params}')


In [39]:
q3 = """
SELECT
    produtos.nomeProduto,
    count(ItensPedido.idItem) as total_pedidos,
    sum(ItensPedido.quantidade) as qnt_vendido
 
    from produtos join itensPedido
        on produtos.idProduto = ItensPedido.fkProduto
        group by produtos.nomeProduto
"""
exe_query(q3)

,nomeProduto,total_pedidos,qnt_vendido
0,Caloi Elite Carbon,26,45
1,Caloi Urbam,35,68
2,Capacete Giro Register,35,62
3,Cassete Shimano 11v,39,89
4,Freio a Disco Hidráulico Shimano,36,72
5,Grupo Shimano Deore,37,80
6,Luva Ciclismo Gel,35,62
7,Luz Traseira Recarregável,28,54
8,Oggi Lite Tour,33,72
9,Sense Impact Pro,25,53


In [41]:
q_vw = """
IF OBJECT_ID('vwFatoVendas', 'V') is not null
    drop view vwFatoVendas 

exec('
    create view vwFatoVendas as
    select
        pedidos.idPedido, 
        pedidos.dataPedido,
        clientes.nomeCliente,
        vendedores.nomeVendedor,
        regionais.nomeRegional,
        produtos.nomePrduto,
        LinhasProdutos.nomeLinhaPoduto,
        itensPedido.PrecoUnitario,
        itensPedido.desconto,
        (itensPEdido.quantidade * (
            itenspedido.precoUnitario - itenspedido.desconto)
            ) as valor_total
        
        from pedidos 
        join clientes on pedidos.fkCliente = clientes.idPedido
        join vendedores on pedidos.fkVendedor = vendedores.idVendedor
        join regionais on vendedores.fkRegional = regionais.idReional
        join itensPEdido on itensPedido.fkPedido = pedidos.idPedido
        join produtos on itensPedido.fkProduto = produtos.idProduto
        join LinhasProdutos on produtos.fkLinha = LinhasProdutos.idLinhaProduto

')
"""

cusror = conn.cursor()
cusror.execute(q_vw)
conn.commit()

df_fatovendas = pd.read_sql('select * from vwFatoVendas', conn)

df_fatovendas.columns

: 

In [ ]:
# from pyspark.sql import SparkSession

# spark: SparkSession = SparkSession.builder \
#     .master("local[2]") \
#     .appName("Teste") \
#     .getOrCreate()

# spark.stop()